In [80]:
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import ttest_ind


# Archivo separado por comas
df_webdata = pd.read_csv('df_webdata.csv')
df_final = pd.read_csv('df_final_clients.csv')

# Filtrar df_webdata para crear df_participants:
df_participants = df_webdata[df_webdata['variation'] != "Not participant"]

# Verificar las primeras filas del nuevo DataFrame
df_participants.head()

,client_id,visit_id,duration,start,step_1,step_2,step_3,confirm,variation
3,555,637149525_38041617439_716659,00:02:38,1,1,1,1,1,Test
4,647,40369564_40101682850_311847,00:06:17,1,1,1,1,1,Test
11,934,7076463_57954418406_971348,00:02:22,4,0,0,0,0,Test
12,1028,557292053_87239438319_391157,00:08:58,1,5,2,1,0,Control
14,1104,543158812_46395476577_767725,00:00:00,1,0,0,0,0,Control


In [81]:
#Hipótesis Nula (H0): No hay diferencia en el completion rate entre los grupos.
#Hipótesis Alternativa (H1): Hay una diferencia significativa en el completion rate entre los grupos.

In [82]:
def paso_completado(val):
    return 1 if val > 0 else 0

# Crear la nueva columna 'completion' sumando los pasos binarizados
#df_participants.loc[:, 'completion'] = (
#    df_participants['start'].apply(paso_completado) +
#    df_participants['step_1'].apply(paso_completado) +
#    df_participants['step_2'].apply(paso_completado) +
#    df_participants['step_3'].apply(paso_completado) +
#    df_participants['confirm'].apply(paso_completado) )

df_participants.loc[:, 'completion'] = (df_participants['confirm'].apply(paso_completado))

# Ahora, para calcular el completion rate, filtramos los que completaron los 5 pasos
completion_rate_control = df_participants[(df_participants['variation'] == 'Control') & (df_participants['completion'] == 1)].shape[0] / df_participants[df_participants['variation'] == 'Control'].shape[0] * 100
completion_rate_test = df_participants[(df_participants['variation'] == 'Test') & (df_participants['completion'] == 1)].shape[0] / df_participants[df_participants['variation'] == 'Test'].shape[0] * 100

print(f'Completion Rate Grupo Control: {completion_rate_control}%')
print(f'Completion Rate Grupo Test: {completion_rate_test}%')

Completion Rate Grupo Control: 49.89610147939088%
Completion Rate Grupo Test: 58.571658961401994%


C:\Users\Usuario\AppData\Local\Temp\ipykernel_18056\3791403091.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_participants.loc[:, 'completion'] = (df_participants['confirm'].apply(paso_completado))


In [83]:
# Calcular el número de éxitos (aquellos que completaron los 5 pasos) y el total de cada grupo
exitos_control = df_participants[(df_participants['variation'] == 'Control') & (df_participants['completion'] == 1)].shape[0]
total_control = df_participants[df_participants['variation'] == 'Control'].shape[0]

exitos_test = df_participants[(df_participants['variation'] == 'Test') & (df_participants['completion'] == 1)].shape[0]
total_test = df_participants[df_participants['variation'] == 'Test'].shape[0]

In [84]:


# Lista de éxitos y totales
exitos = [exitos_control, exitos_test]
totales = [total_control, total_test]

# Prueba de proporciones
stat, p_value = proportions_ztest(exitos, totales)

# Mostrar los resultados
print(f'Estadístico z: {stat:.4f}')
print(f'Valor p: {p_value:.4f}')

# El estadístico z indica la distancia, en unidades de desviación estándar, 
# de la diferencia observada respecto a la hipotesis nula de que no hay diferencia.
# Como sale muy grande en valores absolutos, indica que la diferencia en los 
# completion rates es mucho mayor de lo que esperaríamos si no hubiera diferencia real.

# El valor p te dice la probabilidad de observar la diferencia si la hipótesis nula 
# fuera verdadera. Como es muy pequeño, se puede concluir que hay una diferencia 
# estadísticamente significativa.

# El resultado es estadísticamente significativo y además la diferencia en 
# completion rate es más del 5%, podemos afirmar con mayor confianza que las 
# modificaciones realizadas en tu experimento tuvieron un impacto positivo.

Estadístico z: -22.8989
Valor p: 0.0000


In [85]:
#Hipótesis Nula (H0): No hay diferencia en la duracion entre los grupos.
#Hipótesis Alternativa (H1): Hay una diferencia significativa en la duracion entre los grupos.

In [86]:
# Filtrar los datos para solo aquellos con completion == 1 y hacer una copia
completed_sessions = df_participants[df_participants['completion'] == 1].copy()

# Asegurarse de que no hay valores nulos antes de dividir, usando loc
completed_sessions.loc[:, 'duration'] = completed_sessions['duration'].fillna('00:00:00')

# Dividir la columna 'duration' en tres columnas
time_split = completed_sessions['duration'].str.split(':', expand=True)

# Asignar cada parte a una columna usando loc
completed_sessions.loc[:, 'h'] = time_split[0].astype(int)  # Columna de horas convertida a entero
completed_sessions.loc[:, 'm'] = time_split[1].astype(int)  # Columna de minutos convertida a entero
completed_sessions.loc[:, 's'] = time_split[2].astype(int)  # Columna de segundos convertida a entero

# Crear la nueva columna 'duration_s' calculando el total de segundos
completed_sessions['duration_s'] = completed_sessions['h'] * 3600 + completed_sessions['m'] * 60 + completed_sessions['s']
completed_sessions.drop(columns=['h', 'm', 's'], inplace=True)


completed_sessions.head()

,client_id,visit_id,duration,start,step_1,step_2,step_3,confirm,variation,completion,duration_s
3,555,637149525_38041617439_716659,00:02:38,1,1,1,1,1,Test,1,158
4,647,40369564_40101682850_311847,00:06:17,1,1,1,1,1,Test,1,377
18,1195,393817425_39015278493_996341,00:04:05,1,1,1,1,1,Control,1,245
19,1197,71862471_21202285428_848395,00:01:35,1,1,2,2,1,Control,1,95
21,1336,583743392_96265099036_939815,00:03:31,1,1,1,1,1,Test,1,211


In [ ]:
# Separar las duraciones de las sesiones por grupo
duraciones_control = completed_sessions[completed_sessions['variation'] == 'Control']['duration_s']
duraciones_test = completed_sessions[completed_sessions['variation'] == 'Test']['duration_s']

# Calcular la media de las duraciones en cada grupo
media_duracion_control = duraciones_control.mean()
media_duracion_test = duraciones_test.mean()

# Mostrar las medias
print(f'Media de duración en Grupo Control: {media_duracion_control:.2f} segundos')
print(f'Media de duración en Grupo Test: {media_duracion_test:.2f} segundos')

# Realizar la prueba t
t_stat, p_value = ttest_ind(duraciones_control, duraciones_test, equal_var=False) # Usar equal_var=False si no asumimos varianzas iguales

# Mostrar los resultados
print(f'Estadístico t: {t_stat:.4f}')
print(f'Valor p: {p_value:.4f}')

# El valor t es bastante alto y el valor p muy pequeño, <0.05, lo que indica que la diferencia 
# en el tiempo promedio de sesión entre los grupos de control y test es estadísticamente significativa. 
# Esto implica que es muy probable que alguna característica del grupo de test haya influenciado 
# en la duración de las sesiones.

Media de duración en Grupo Control: 391.45 segundos
Media de duración en Grupo Test: 360.03 segundos
Estadístico t: 5.9555
Valor p: 0.0000
